In [1]:
import cv2
import numpy as np
import os
import matplotlib
%matplotlib inline

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks.callbacks import ModelCheckpoint
from keras.layers.pooling import AveragePooling2D, MaxPooling2D
import tensorflow as tf
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, Convolution2D, BatchNormalization
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from imutils import paths
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import argparse
import pickle
import cv2
import os   
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import accuracy_score, recall_score, precision_score
from keras.models import load_model

Using TensorFlow backend.


In [3]:
from sklearn.linear_model import LogisticRegression

In [ ]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

### Prepare data

In [2]:
path_train = '../Datasets/Videos/lstm/train/'
path_test = '../Datasets/Videos/lstm/test/'

In [48]:
X_train = []
X_test = []

y_train = []
y_test = []

In [49]:
for file in os.listdir(path_train):
    if file.endswith(".csv"):
        data = pd.read_csv(path_train + file).drop('Unnamed: 0', axis=1)
        pos = 9
        while pos < data.shape[0]:
            X_train.append(data.drop('label', axis=1).values[pos-9: pos+1, :].flatten())
            y_train.append(data['label'].iloc[pos])
            pos += 1

In [50]:
for file in os.listdir(path_test):
    if file.endswith(".csv"):
        data = pd.read_csv(path_test + file).drop('Unnamed: 0', axis=1)
        pos = 9
        while pos < data.shape[0]:
            X_test.append(data.drop('label', axis=1).values[pos-9: pos+1, :].flatten())
            y_test.append(data['label'].iloc[pos])
            pos += 1

In [51]:
X_train = np.array(X_train)
X_test = np.array(X_test)

def label_to_float(x):
    return 0.0 if x == 'fire' else 1.0

y_train = np.array([label_to_float(x) for x in y_train])
y_test = np.array([label_to_float(x) for x in y_test])

In [59]:
scale = np.abs(X_train).max()

X_train /= scale
X_test /= scale

In [52]:
y_train.shape

(1676,)

### Logistic regression

In [60]:
model = LogisticRegression(n_jobs=4, verbose=1)
model.fit(X_train, y_train)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   18.6s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=4, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)

In [61]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred, pos_label=0))
print("Recall:", recall_score(y_test, pred, pos_label=0))
print("F1:", f1_score(y_test, pred, pos_label=0))
print("ROC AUC:", roc_auc_score(y_test, pred))

Accuracy: 0.7971014492753623
Precision: 0.9678899082568807
Recall: 0.8115384615384615
F1: 0.8828451882845189
ROC AUC: 0.6870192307692308


### Random forest

In [62]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_jobs=4, verbose=1)
forest.fit(X_train, y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    3.7s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                       oob_score=False, random_state=None, verbose=1,
                       warm_start=False)

In [63]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

pred = forest.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred, pos_label=0))
print("Recall:", recall_score(y_test, pred, pos_label=0))
print("F1:", f1_score(y_test, pred, pos_label=0))
print("ROC AUC:", roc_auc_score(y_test, pred))

Accuracy: 0.6956521739130435
Precision: 0.9680851063829787
Recall: 0.7
F1: 0.8125
ROC AUC: 0.6625


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


### SVM

In [64]:
from sklearn.svm import SVC

svc = SVC(verbose=1)
svc.fit(X_train, y_train)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=1)

In [65]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

pred = svc.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred, pos_label=0))
print("Recall:", recall_score(y_test, pred, pos_label=0))
print("F1:", f1_score(y_test, pred, pos_label=0))
print("ROC AUC:", roc_auc_score(y_test, pred))

Accuracy: 0.9166666666666666
Precision: 0.9540229885057471
Recall: 0.9576923076923077
F1: 0.9558541266794626
ROC AUC: 0.6038461538461538
